<a href="https://colab.research.google.com/github/sebatlab/Antaki2021/blob/main/make_tada_input_2021_11_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# De Novo SNV/INDEL
# df_dnm = pd.read_csv("/content/dn_snv_indel.2021_11_17.tsv", sep = "\t") # 14993 rows × 16 columns
df_dnm = pd.read_csv("/content/drive/My Drive/antaki2021_manuscript/brainspan/dn_snv_indel.2021_11_17.tsv", sep = "\t").drop_duplicates() # 14993 rows × 16 columns

# LOF Counts
df_dnm_lof = df_dnm[(df_dnm["phenotype"] == "ASD") & (df_dnm["loss_of_function"] == 1)][["gene", "loss_of_function"]].reset_index(drop = True)
df_dnm_lof_count = df_dnm_lof.groupby(["gene"], as_index = False).count()

# MIS Counts
df_dnm_mis = df_dnm[(df_dnm["phenotype"] == "ASD") & (df_dnm["missense"] == 1)][["gene", "missense"]].reset_index(drop = True)
df_dnm_mis_count = df_dnm_mis.groupby(["gene"], as_index = False).count()

In [13]:
# Private, inherited SNV/INDEL
# df_inh = pd.read_csv("/content/drive/My Drive/antaki2021_manuscript/brainspan/reach.spark.ssc.combined.private.inherited.lof.mis.txt", sep = "\t") # 1069029 rows × 17 columns
df_inh = pd.read_csv("/content/drive/My Drive/antaki2021_manuscript/brainspan/reach.spark.ssc.combined.private.inherited.lof.mis.txt", sep = "\t").drop_duplicates()
df_inh["t_asd"] = df_inh["aff_pt"] + df_inh["aff_mt"]
df_inh["nt_asd"] = df_inh["aff_pn"] + df_inh["aff_mn"]

# LOF Counts
df_inh_lof = df_inh[df_inh["lof"] == 1][["gene", "lof", "t_asd", "nt_asd"]].reset_index(drop = True)
df_inh_lof_sum = df_inh_lof.groupby(["gene"], as_index = False)["t_asd", "nt_asd"].sum()

# MIS Counts
df_inh_mis = df_inh[df_inh["mis"] == 1][["gene", "mis", "t_asd", "nt_asd"]].reset_index(drop = True)
df_inh_mis_sum = df_inh_mis.groupby(["gene"], as_index = False)["t_asd", "nt_asd"].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if sys.path[0] == '':


In [14]:
tada_lof_col = {
    'gene':'gene.id',
    'loss_of_function' : 'dn.cls1',
    't_asd': 'trans.cls1',
    'nt_asd': 'ntrans.cls1'
}

tada_mis_col = {
    'gene':'gene.id',
    'missense' : 'dn.cls2',
    't_asd': 'trans.cls2',
    'nt_asd': 'ntrans.cls2'
}

df_tada_lof = df_dnm_lof_count.merge(df_inh_lof_sum, on = "gene", how = "outer").fillna(0).rename(columns = tada_lof_col)
df_tada_mis = df_dnm_mis_count.merge(df_inh_mis_sum, on = "gene", how = "outer").fillna(0).rename(columns = tada_mis_col)
df_tada = df_tada_lof.merge(df_tada_mis, on = "gene.id", how = "outer").fillna(0)
df_tada

,gene.id,dn.cls1,trans.cls1,ntrans.cls1,dn.cls2,trans.cls2,ntrans.cls2
0,AAAS,2.0,6.0,10.0,0.0,33.0,45.0
1,AADACL4,1.0,4.0,4.0,0.0,27.0,32.0
2,AASDH,1.0,12.0,10.0,0.0,65.0,55.0
3,ABCA5,1.0,12.0,19.0,1.0,106.0,97.0
4,ABCC5,1.0,5.0,9.0,4.0,71.0,56.0
...,...,...,...,...,...,...,...
21355,ZNF83,0.0,0.0,0.0,0.0,35.0,37.0
21356,ZNRD1,0.0,0.0,0.0,0.0,5.0,3.0
21357,ZPR1,0.0,0.0,0.0,0.0,18.0,31.0
21358,"ZSCAN26,ZSCAN26,AL021997.3,AL021997.3",0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
df_mutation_rates = pd.read_csv("/content/drive/My Drive/antaki2021_manuscript/brainspan/samocha_mut_rates.txt", sep = "\t")
df_mutation_rates["lof_rate"] = 10**df_mutation_rates["lof"]
df_mutation_rates["mis_rate"] = 10**df_mutation_rates["mis"]

In [16]:
df = df_tada.merge(df_mutation_rates, left_on='gene.id', right_on='gene', how='inner').rename(columns={'mis_rate':'mut.cls2','lof_rate':'mut.cls1'})

final_col = [
    'gene.id',
    'mut.cls1',
    'dn.cls1',
    'trans.cls1',
    'ntrans.cls1',
    'mut.cls2',
    'dn.cls2',
    'trans.cls2',
    'ntrans.cls2'
]

df = df[final_col]


df[df['gene.id'] == 'SYNGAP1']

df.to_csv("tada_table_2021_11_18.tsv", sep = '\t', index = False)

In [ ]:
# run this after running TADA to get the final results table (which we then merge with input table)
df_in  = pd.read_csv("/content/tada_table_2021_11_18.tsv", sep = "\t")
df_out = pd.read_csv("/content/tada_results_2021_11_18.tsv", sep = "\t")

df_in = df_in.rename(columns = 
                     {"gene.id": "gene", 
                      "dn.cls1": "dn_lof",
                      "trans.cls1": "trans_lof",
                      "ntrans.cls1": "ntrans_lof",
                      "dn.cls2": "dn_mis",
                      "trans.cls2": "trans_mis",
                      "ntrans.cls2": "ntrans_mis"})

df_out = df_out.rename(columns = 
                       {"BF.cls1": "bayes_factor_lof",
                        "BF.cls2": "bayes_factor_mis",
                        "BF.total": "bayes_factor"
                       })

df_in = df_in[["gene", 
               "dn_lof", "trans_lof", "ntrans_lof",
               "dn_mis", "trans_mis", "ntrans_mis"]]

# df_in # 16729 rows × 7 columns
# df_out # 16729 rows × 5 columns

df_tada_inputs_and_outputs = df_in.merge(df_out, on = "gene")
# df_tada_inputs_and_outputs # 16729 rows × 11 columns
df_tada_inputs_and_outputs = df_tada_inputs_and_outputs.sort_values(by = 'qval', ascending = True).reset_index(drop = True)
df_tada_inputs_and_outputs.to_csv("tada_input_and_results_2021_11_18.csv", sep = '\t', index = False)

# TADA gene list is all genes with qval < 0.05
df_tada_inputs_and_outputs_qval_filter = df_tada_inputs_and_outputs[df_tada_inputs_and_outputs["qval"] < 0.05]
df_tada_inputs_and_outputs_qval_filter.to_csv("tada_input_and_results_2021_11_18.qval_0.05_.csv", sep = '\t', index = False)
